In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink
import seaborn as sns

import statsmodels.api as sm

from basic_tools import *

"""

jupyter nbconvert 5_association.ipynb --to script
python 5_association.py albumin 2 0

for i in {00..101};do python 5_association.py $i 2 1;done


for i in {00..10};do python 5_association.py $i 2 0;done
for i in {11..20};do python 5_association.py $i 2 0;done
for i in {21..30};do python 5_association.py $i 2 0;done
for i in {31..40};do python 5_association.py $i 2 0;done
for i in {41..50};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {61..70};do python 5_association.py $i 2 0;done
for i in {71..80};do python 5_association.py $i 2 0;done
for i in {91..90};do python 5_association.py $i 2 0;done
for i in {91..101};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done
for i in {51..60};do python 5_association.py $i 2 0;done

for i in {00..101};do python 5_association.py $i 1 0,2;done
for i in {00..101};do python 5_association.py $i 1 1;done

for i in {00..20};do python 5_association.py $i 1 1;done
for i in {21..40};do python 5_association.py $i 1 1;done
for i in {41..60};do python 5_association.py $i 1 1;done
for i in {61..80};do python 5_association.py $i 1 1;done
for i in {81..101};do python 5_association.py $i 1 1;done

"""

'\n\njupyter nbconvert 5_association.ipynb --to script\npython 5_association.py albumin 2 0\n\nfor i in {00..101};do python 5_association.py $i 2 1;done\n\n\nfor i in {00..10};do python 5_association.py $i 2 0;done\nfor i in {11..20};do python 5_association.py $i 2 0;done\nfor i in {21..30};do python 5_association.py $i 2 0;done\nfor i in {31..40};do python 5_association.py $i 2 0;done\nfor i in {41..50};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {61..70};do python 5_association.py $i 2 0;done\nfor i in {71..80};do python 5_association.py $i 2 0;done\nfor i in {91..90};do python 5_association.py $i 2 0;done\nfor i in {91..101};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\nfor i in {51..60};do python 5_association.py $i 2 0;done\n\nfor i in {00..101};do python 5_association.py $i 1 0,2;done\nfor i in {00..101};do py

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [3]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=phenotypes.columns.difference(['age','sex','cohort','diabetes'])

In [4]:
len(binary_continuous_traits)

97

In [48]:
def get_a1_freq_case_control(row):
    dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(row['marker_name']).astype(float)
    dosage[dosage==-1]=np.nan
    
    if plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A1'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A2']:
        pass
    elif plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A2'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A1']:
        dosage=2-dosage

    dosage_control=dosage[pheno['pheno']==1]
    dosage_case=dosage[pheno['pheno']==2]
    
    a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*((~np.isnan(dosage_case)).sum()))
    a1_freq_control=(2*(dosage_control==2).sum()+1*(dosage_control==1).sum())/(2*((~np.isnan(dosage_control)).sum()))

    return a1_freq_case,a1_freq_control
    
def get_a1_freq_case_nan(row):
    dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(row['marker_name']).astype(float)
    dosage[dosage==-1]=np.nan
    
    if plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A1'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A2']:
        pass
    elif plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a1']==row['A2'] and plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[row['marker_name']]['a2']==row['A1']:
        dosage=2-dosage

    dosage_case=dosage[pheno['pheno']!=-9]

    a1_freq_case=(2*(dosage_case==2).sum()+1*(dosage_case==1).sum())/(2*((~np.isnan(dosage_case)).sum()))

    return a1_freq_case,np.nan

In [46]:
def name_to_pos(name):
    if name[:3]=='AA_':
        return int(name.split('_')[3])
    elif name[:4]=='HLA_':
        return plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_'+name.split('*')[0].split('_')[1]+'*')].iloc[0]['pos']
    else:
        raise

In [7]:
SNP_DB=pd.read_csv('data/mart_export_SNP.txt',sep='\t')

In [8]:
SNP_DB['Variant alleles set']=SNP_DB['Variant alleles'].apply(lambda x: set(x.split('/')).difference(['-']))

In [9]:
rsid_mapper=plink_KCHIP_HLA_AA_SNP_1000G_bim.copy()
rsid_mapper=rsid_mapper[(~rsid_mapper.index.str.contains('HLA'))
                        |~(rsid_mapper.index.str.contains('AA_'))]

In [10]:
def pos_a1_a2_to_rsid(row):
    
    SNP_DB_match=SNP_DB[(SNP_DB['Chromosome/scaffold position start (bp)']<=row['pos'])&
                        (SNP_DB['Chromosome/scaffold position end (bp)']>=row['pos'])]
    if len(SNP_DB_match)==0:
        return row.name
    
    SNP_DB_match_match=SNP_DB_match[SNP_DB_match['Variant alleles set'].apply(lambda x: set([row['a1'],row['a2']]).issubset(x))]
    if len(SNP_DB_match_match)==0:
        return row.name
    elif len(SNP_DB_match_match)==1:
        return SNP_DB_match_match.iloc[0]['Variant name']

    SNP_DB_match_match_match=SNP_DB_match_match[(SNP_DB_match_match['Chromosome/scaffold position start (bp)']<=row['pos'])&
                            (SNP_DB_match_match['Chromosome/scaffold position end (bp)']>=row['pos'])]        
    if len(SNP_DB_match_match_match)==0:
        return SNP_DB_match_match.iloc[0]['Variant name']
    elif len(SNP_DB_match_match_match)==1:
        return SNP_DB_match_match_match.iloc[0]['Variant name']
    else:
        SNP_DB_match_match_match.iloc[0]['Variant name']

In [11]:
rsid_mapper['marker_name_pub']=rsid_mapper.apply(pos_a1_a2_to_rsid,axis=1)

In [12]:
def marker_name_to_marker_name_pub(marker_name):
    if marker_name[:4]=='HLA_':
        return marker_name.replace('_','-')
    elif marker_name[:3]=='AA_':
        marker_name_split=marker_name.split('_')
        return 'HLA-{} position {}'.format(marker_name_split[1],marker_name_split[2])
    else:
        return rsid_mapper.loc[marker_name]['marker_name_pub']

In [49]:
for phenotype_name in binary_continuous_traits:
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    
    phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'    
    print(phenotype_name,phenotype_type)
    try:
        for step_idx_sub in range(1,100):
            print(step_idx_sub,end=' ')
            if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
                print('meets end',step_idx_sub)
                break

            GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
            GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)

            plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
            plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})
            plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
            
            if phenotype_type=='binary':
                assert set(pheno['pheno'].unique())=={-9,1,2}
                a1_freq_case_control=plink_result_munge.apply(get_a1_freq_case_control,axis=1)
                plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
                plink_result_munge['A1_freq_control']=[i[1] for i in a1_freq_case_control]
            else:
                a1_freq_case_nan=plink_result_munge.apply(get_a1_freq_case_nan,axis=1)
                plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_nan]
                plink_result_munge['A1_freq_control']=np.nan
            
            plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
            plink_result_munge['note']='unphased bialleic'

            result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)
            result_merge['phenotype_name']=phenotype_name
            
            result_merge['marker_name_pub']=result_merge['marker_name'].apply(marker_name_to_marker_name_pub)
            
            if phenotype_type=='binary':
                result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
            else:
                result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

            result_merge['step']=step_idx_sub
            with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
                result_merge['condition']=','.join(f.read().split())        
            result_merge=result_merge[['phenotype_name','samples(case/control)','step','condition','marker_name','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
            result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')
    except:
        print('error',phenotype_name,step_idx_sub)

FEV_over_FVC_predicted continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
FEV_predicted continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


2 

...

2 meets end 2
sleep_duration continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
small_baby binary
1 2 meets end 2
stomach_cancer binary
1 2 meets end 2
stroke binary
1 2 meets end 2
systolic_blood_pressure continuous
1 2 3 meets end 3
t2_diabetes binary
1 2 3 meets end 3
thyroid_cancer binary
1 2 meets end 2
thyroid_disease binary
1 2 3 4 meets end 4
total_bilirubin continuous
1 2 meets end 2
total_cholesterol continuous
1 2 3 meets end 3
transient_ischemic_attacks binary
1 2 meets end 2
triglyceride continuous
1 2 3 4 meets end 4
tsh continuous
1 

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in long_scalars


2 meets end 2
tuberculosis binary
1 2 meets end 2
urinary_tract_infection binary
1 2 meets end 2
waist_circumference continuous
1 2 meets end 2
wbc continuous
1 2 3 4 5 6 meets end 6
weight continuous
1 2 3 4 5 6 meets end 6


In [7]:
def name_to_pos(name):
    if name[:3]=='AA_':
        return int(name.split('_')[3])
    elif name[:4]=='HLA_':
        return plink_KCHIP_HLA_AA_SNP_1000G_bim.loc[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_'+name.split('*')[0].split('_')[1]+'*')].iloc[0]['pos']
    else:
        raise

In [18]:
GAT_result['marker_name'].str.slice(stop=3).unique()

array(['AA_', 'AFF', 'HLA'], dtype=object)

In [20]:
GAT_result['marker_name'].str.slice(stop=3).unique()

array(['AA_', 'HLA'], dtype=object)

In [ ]:
for phenotype_name in binary_continuous_traits:
    phenotype_name='height'
    
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    
    phenotype_type='binary' if len(pheno['pheno'][pheno['pheno']!=-9].value_counts())<3 else 'continuous'    
    print(phenotype_name,phenotype_type)
    
    for step_idx_sub in range(1,100):
        print(step_idx_sub,end=' ')
        if os.path.exists(data_out_assoc_phenotype_path+'step_{:02d}.cond.stop'.format(step_idx_sub)):
            print('meets end',step_idx_sub)
            break

        GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx_sub),sep='\t')
        GAT_result['POS']=GAT_result['marker_name'].map(name_to_pos)

        plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx_sub,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
        plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','T_STAT':'Z','Z_STAT':'Z','OBS_CT':'nobs'})
        plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
        if phenotype_type=='binary':
            assert set(pheno['pheno'].unique())=={-9,1,2}
            a1_freq_case_control=plink_result_munge.apply(get_a1_freq_case_control,axis=1)
            plink_result_munge['A1_freq_case']=[i[0] for i in a1_freq_case_control]
            plink_result_munge['A1_freq_control']=[i[1] for i in a1_freq_case_control]
        else:
            plink_result_munge['A1_freq_case'],plink_result_munge['A1_freq_control']=np.nan,np.nan
        plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
        plink_result_munge['note']='unphased bialleic'

        result_merge=pd.concat([GAT_result,plink_result_munge],sort=True)
        result_merge['phenotype_name']=phenotype_name
        if phenotype_type=='binary':
            result_merge['samples(case/control)']='{}/{}'.format((pheno['pheno']==2).sum(),(pheno['pheno']==1).sum())
        else:
            result_merge['samples(case/control)']='{}'.format((pheno['pheno']!=-9).sum())

        result_merge['step']=step_idx_sub
        with open(data_out_assoc_phenotype_path+'step_{:02d}.cond'.format(step_idx_sub)) as f:
            result_merge['condition']=','.join(f.read().split())        
        result_merge=result_merge[['phenotype_name','samples(case/control)','step','condition','marker_name','note','term','POS','A1','A2','A1_freq_case','A1_freq_control','multi_allele','nobs','Z','coef','std','chisq','df','P']]
        result_merge.sort_values('POS').to_csv(data_out_assoc_phenotype_path+'step_{:02d}.merge.result.tsv'.format(step_idx_sub),sep='\t')


,Variant name,Variant source,Chromosome/scaffold name,Chromosome/scaffold position start (bp),Chromosome/scaffold position end (bp),Variant alleles
1402001,rs76863928,dbSNP,6,34207474,34207474,T/A
1402002,rs869142459,dbSNP,6,34207474,34207474,T/-
1402003,rs1554142463,dbSNP,6,34207474,34207474,T/AA
